In [7]:
# This cell parses data from the CSV and should (in theory) push it to AWS
# For now it just creates the array raw_questions, which holds all questions
# from the contributions files.

import csv

# initializing the titles and rows list 
fields = [] 
rows = [] 
raw_questions = []

filename = 'contributions.csv'
# reading csv file 
with open(filename, 'r', encoding="utf8") as csvfile: 
    # creating a csv reader object 
    csvreader = csv.reader(csvfile) 
      
    # extracting field names through first row 
    fields = next(csvreader) 
  
    # extracting each data row one by one 
    for row in csvreader: 
        rows.append(row) 

    curr_question = ""
    curr_text = ""
    curr_topic = ""
    curr_folder = ""
    curr_answer = ""
    curr_number = 0
    curr_follow_up = 0
    endorsed = 0
    
    # This will help us take the most recent version because it always comes first
    already_filled = 0
    for row in rows[:]: 
        # parsing each column of a row 
        for col,field in zip(row, fields): 
            if(field == "Post Number" and curr_number != int(col)):
                if(curr_question != "" and curr_answer != ""):
                    if(curr_folder == 'hw4'):
                        raw_questions.append(curr_question)
                curr_number = int(col)
                curr_follow_up = 0
                curr_question = ""
                already_filled = 0
                curr_answer = ""
                curr_topic = ""
                endorsed = 0
                
            elif(field == "Submission HTML Removed"): #the question could also be in subject
                curr_text = col
                
            elif(field == "Subject"): #We want to check if this is actually the question eventaully
                curr_topic = col
                
            elif(field == "Folders"):
                curr_folder = col
                
            elif(field == "Endorsed by Instructor"):
                if(col == "TRUE"):
                    endorsed = 1
                else:
                    endorsed = 0
            elif(field == "Part of Post"):
                if(col == "started_off_question"):
                    if(already_filled == 0):
                        curr_question = curr_text
                    already_filled = 0
                elif(col == "updated_question"):
                    # update question entry to post and follow up numbers
                    if(already_filled == 0):
                        curr_question = curr_text
                        already_filled = 1
                        
                elif(col == "updated_i_answer"):
                    # update last entry to the post and follow up number
                    # what if it was appended? how do we update just the end?
                    if(already_filled == 0):
                        curr_answer = curr_answer + "\n" + curr_text
                        already_filled = 1
                    endorsed = 1
                elif(col == "started_off_i_answer"):
                    if(already_filled == 0):
                        curr_answer = curr_answer + "\n" + curr_text
                    already_filled = 0
                    endorsed = 1
                    
                elif(col == "updated_s_answer"):
                    # update last entry to the post and follow up number
                    # what if it was appended? how do we update just the end?
                    if(already_filled == 0):
                        curr_answer = curr_answer + "\n" + curr_text
                        already_filled = 1
                elif(col == "started_off_s_answer"):
                    # append to answer - if it was previously endorsed and were appending should we keep it endorsed?
                    if(already_filled == 0):
                        curr_answer = curr_answer + "\n" + curr_text
                    already_filled = 0
                    
                elif(col == "followup"):
                    # new entry for the new followup question 
                    if(curr_folder == 'hw4'):
                        raw_questions.append(curr_question)
                        curr_question = ""
                        curr_answer = ""
                        endorsed = 0
                        already_filled = 0
                        
                    curr_follow_up = curr_follow_up + 1
                    curr_question = curr_text
                elif(col == "reply_to_followup"):
                    curr_answer = curr_answer + "\n" + curr_text
                    
num = 0
for i in raw_questions:
    print(num)
    print(i)
    print('\n')
    # Probably need a better way to do this
    if not i:
        raw_questions.pop(num)
    if i == '\n\n':
        raw_questions.pop(num)
    num = num+1

#myDB.close()


0
For my let reverse, I have:

let reverse (l: 'a list) : 'a list =
  fold (fun x acc -&gt; acc @ x) [] l

But codio told me:
This expression has type 'a list but an expression was expected of type 'a list list The type variable 'a occurs inside 'a list.

However, my function should return a' list. Also this function is not related to other functions, the error should be within this?


1
When will homework 4 be released?


2
Are we allowed to use functions like List.map on problem 1 of homework 4?


3
I dont know if anyone else had this, but when I opened my hw04(i haven't done anything to it yet), I got error on assert(one the ;; open assert) saying it is an unbound modulus. What should I do about it?


4
I am getting the same error on assert, however the file complies and builds perfectly. 


5
Thanks, mine is working now!


6
I am getting the same error message and they won't go away after clean and build. What should I do?


7
We learned how the transform and fold functions work in

I did read that question and I'm not sure how to apply it because my code doesn't have any if-statements and I have semi colons after my commands. I know this is kind of a vague question but I'm just not sure what the error means


605
let delete_last (v: 'a) (q: 'a deque) : unit =
  if not (valid q) then failwith "delete_last: given invalid deque";
  let rec loop (n: 'a dqnode option) : unit = begin match n with 
    |None -&gt; ()
    |Some(x) -&gt; begin match x.prev with
        |None -&gt; ()
        |Some(x2) -&gt; if x2.v = v then (x.prev &lt;- x2.prev; let update = begin match x2.prev with
            |None -&gt; q.head &lt;- Some x
            |Some(x3) -&gt; x3.next &lt;- Some x
          end in
          update;) else loop x.prev  
      end
  end in 
  begin match q.tail with
    |None -&gt; ()
    |Some(x) -&gt; if x.v = v then remove_tail q
      else loop q.tail
  end

File "deque.ml", line 1:Error: The implementation deque.ml does not match the interface deque.cmi: Valu

In [ ]:
# This cell runs TF-IDF using the Stack Exchange Archives and then shortens
# all questions to 25 words so that they are prepared for BURT


# Just using questions for right now

In [13]:
from bert_serving.client import BertClient  
bc = BertClient()  
doc_vecs = bc.encode(raw_questions)

In [19]:
# This cell allows you to type a question and find semantically similar questions using BURT, eventually
# we should run this for every question in raw_questions

import np
print(doc_vecs)
topk = 10
query = input('your question: ')
query_vec = bc.encode([query])[0]
# compute simple dot product as score
score = np.sum(query_vec * doc_vecs, axis=1)
topk_idx = np.argsort(score)[::-1][:topk]
for idx in topk_idx:
    print('> %s\t%s' % (score[idx], raw_questions[idx]))

[[ 0.2603337   0.26356497  0.7432169  ...  0.58039534  0.21430358
   0.8758928 ]
 [ 0.04027047  0.1673576   0.04610902 ...  0.7403443  -0.25136518
   0.39421248]
 [ 0.61585414  0.0269997   0.23100404 ...  0.5974973  -0.093358
   0.31836292]
 ...
 [ 0.3507512   0.36690938  0.30640066 ...  0.4945316   0.0163654
   0.65210605]
 [ 0.43843868  0.12522021  0.1926335  ...  0.5394593   0.07878795
   0.65595716]
 [ 0.22471753  0.05150049  0.20319511 ...  0.65726227 -0.15516265
   0.5353097 ]]
your question: truncate recursive
> 493.08865	bodyloop is checking for the element in the body of the queue (besides the head)
> 489.88745	Good Afternoon Instructors

For the contains function in linkedQueue, I have:

  let contains (elt: 'a) (q: 'a queue) : bool =    if not (valid q) then failwith "contains: given invalid queue";    let rec loop (no: 'a qnode option) (my_bool: bool) : bool =      begin match no with      | None -&gt; my_bool      | Some n -&gt; loop n.next (n.v == elt || my_bool)    end  